# This script annotates transcripts of video simulation role-playing exercises from job candidates using large language models

In [10]:
# Install libraries that might not be installed as necessary (adjust manually) through Anaconda prompt

In [29]:
# Import necessary libraries
import openai          # OpenAI API for AI model access
from openai import OpenAI  # The main client class for interacting with OpenAI's API
import requests        # HTTP library for making web requests (API calls, downloading data, etc.)
import re              # Regular expressions
import os              # Operating system interface
import pandas as pd    # DataFrames for tabular data manipulation and analysis
import numpy as np     # Numerical computing
import time            # Time-related functions
import random          # Random number generation
import getpass         # pop-up window for API completion
from tqdm import tqdm  # for showing progress bars

# set seed for reproducibility
random.seed(1337)  # Sets seed for Python's random module

In [30]:

# Load data
test_data = pd.read_excel("data_gitp - Copy_for_data_exploration.xlsx")
# test_data = test_data.iloc[:2] # keep only first few rows for pilot data analysis purposes
test_data.head()

# # Advisement 2
# advise = pd.read_csv("data_advise_text.csv")
# # Change management 2
# change_manage = pd.read_csv("data_change_manage_text.csv")
# # Team management 2
# team_manage = pd.read_csv("data_team_manage_text.csv")
# # Team work 2
# team_work = pd.read_csv("data_team_work_text.csv")

,VideoSimulatieType,tw_Beoordeling.all,AD_PROBL,AD_CREAT,AD_OORDE,AD_ORGANS,AD_total
0,Advisement_2,"Eerst, alles blijft tussen vier ogen dat wat v...",1.500000,1.500000,1.501667,2.333333,1.708750
1,Advisement_2,"Ja, nee, ik heb zeker wel verstand van deze za...",1.000000,2.000000,2.000000,2.000000,1.750000
2,Advisement_2,"Nou allereerst, PERSON_FIRSTNAME_1 bedankt voo...",3.331667,3.000000,2.835000,2.833333,3.000000
3,Advisement_2,"Goedemorgen China, ja, ik vind het ook leuk om...",3.333333,2.498333,3.166667,3.165000,3.040833
4,Advisement_2,Oh Surina leuk om kennis met jou te maken. Nat...,3.835000,3.835000,4.000000,4.166667,3.959167


# Prompts

## Advisement

In [31]:
# prompts have the following structure: 
# for each competence, the first prompt is about providing verbal explanation (number + explanation),
# the second prompt is about numerical annotation only.

promptAD = {
    # --- Probleemanalyse (Problem analysis) ---
    "AD_PROBL": (
        "AD_PROBL",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

        **Te evalueren competentie:**  
**Probleemanalyse** - Het vermogen van de kandidaat om de complexe, veelzijdige situatie 
die Lara presenteert te ontleden om de kernproblemen, hun onderlinge verbanden en hun relatieve belangrijkheid te identificeren.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De reactie is generiek, vermijdt het probleem, of richt zich op irrelevante bijzaken. 
De kandidaat maakt geen onderscheid tussen hoofd- en bijzaken, maakt geen verbanden, 
en identificeert het kernprobleem niet.
*Gedragsvoorbeelden*: Biedt vage, motiverende clichés ("Ik sta voor je klaar"). 
Geeft generiek advies dat niet gerelateerd is aan Lara's specifieke problemen. Springt naar een voorbarige oplossing zonder analyse. 
De reactie is onsamenhangend en gaat niet in op de essentie van Lara's problemen.
- **2** (Matig): De kandidaat identificeert oppervlakkige problemen maar behandelt ze geïsoleerd. 
De reactie kan problemen opsommen zonder ze te prioriteren of betekenisvolle verbanden te leggen. 
De kern van het probleem wordt slechts gedeeltelijk of incorrect geïdentificeerd.
*Gedragsvoorbeelden*: Erkent dat er "moeilijke keuzes" of "spanningen" zijn. 
Stelt voor om naar "data" te kijken of "een plan te maken" zonder te specificeren hoe. Stelt een eenvoudige volgende stap voor 
(zoals een pilot) maar zonder te analyseren waarom dit geschikt is voor de samenhangende problemen van groei, 
teamconflict en bedrijfswaarden.
- **3** (Voldoende): De kandidaat identificeert de hoofdproblemen (bijv. werkfrustratie, 
projectfout, persoonlijke stress) en begint ze te verbinden (bijv. persoonlijke stress beïnvloedt 
werkprestaties). Ze kunnen onderscheid maken tussen hoofd- en bijzaken maar niet met grote duidelijkheid. 
Het kernprobleem wordt op basisniveau geïdentificeerd (bijv. "je bent overweldigd").
*Gedragsvoorbeelden*: Stelt verhelderende vragen over de opties of het teamconflict. 
Stelt verder onderzoek voor of het maken van een overzicht van voor-/nadelen/risico's. 
Herkent de spanning tussen bedrijfsgroei en bedrijfswaarden. 
Biedt aan om het probleem te structureren voor de volgende vergadering.
- **4** (Goed): De kandidaat onderscheidt duidelijk hoofdproblemen van bijzaken en onderzoekt actief 
de verbanden tussen hen (bijv. hoe over het hoofd gezien worden leidt tot demotivatie, wat zou kunnen 
bijdragen aan projectfouten). Ze identificeren een genuanceerder kernprobleem (bijv. een conflict 
tussen persoonlijke waarden en bedrijfsdruk, of een breuk in teamcommunicatie en vertrouwen).
*Gedragsvoorbeelden*: Peilt naar de eigen prioriteiten en waarden van de kandidaat om de wortel 
van het dilemma te begrijpen. Vraagt naar de onderliggende oorzaken van het teamconflict. Stelt een proces voor om het team 
rond gedeelde doelen en waarden te verenigen. Synthetiseert informatie uit verschillende scènes om 
een coherent beeld van de situatie te vormen.
- **5** (Uitstekend): De kandidaat toont een verfijnd, holistisch begrip. 
Ze integreren naadloos alle aspecten van het probleem (professioneel, persoonlijk, interpersoonlijk) 
en identificeren het diepe, systemische kernprobleem (bijv. een gebrek aan psychologische veiligheid, een mismatch 
tussen bedrijfsstrategie en kernidentiteit, of een behoefte aan proactief leiderschap en ondersteunende structuren). 
Hun analyse onthult niet-voor de hand liggende verbanden en komt bij de fundamentele oorzaak.
*Gedragsvoorbeelden*: Identificeert dat het kernprobleem niet alleen de strategische keuze is, 
maar het onvermogen van het team om een constructief, waarde-gedreven gesprek hierover te voeren. 
Richt zich op het herbouwen van vertrouwen en gedeeld doel als voorwaarde voor het nemen van een beslissing. 
Toont een duidelijke, logische lijn van de symptomen (Lara's stress, de projectfout) 
naar een systemische oorzaak. Stelt een uitgebreide, gefaseerde aanpak voor die zowel 
de directe praktische behoeften als de onderliggende interpersoonlijke/strategische kwesties aanpakt.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Probleemanalyse" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Verdeel scores over de schaal gebaseerd op werkelijke prestatieverschillen.
- Gebruik de volledige 1-5 schaal correct.
- Streef naar gemiddelde: ~3.0 met standaarddeviatie: ~0.6.
- Geef **beide** terug:
  1. De numerieke beoordeling (1-5, mag één decimaal bevatten, bijv. 3.8).
  2. Een korte uitleg (max 80 woorden) die de beoordeling rechtvaardigt.
- Formatteer uw antwoord zoals dit voorbeeld:
  Beoordeling: 4.2 - Uitleg: De deelnemer peilt naar de eigen prioriteiten en waarden van de kandidaat om de wortel van het dilemma te begrijpen.
  Synthetiseert informatie uit verschillende scènes om een coherent beeld van de situatie te vormen.

Transcript:
```{text}```
""",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

        **Te evalueren competentie:**  
**Probleemanalyse** - Het vermogen van de kandidaat om de complexe, veelzijdige situatie 
die Lara presenteert te ontleden om de kernproblemen, hun onderlinge verbanden en hun relatieve belangrijkheid te identificeren.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De reactie is generiek, vermijdt het probleem, of richt zich op irrelevante bijzaken. 
De kandidaat maakt geen onderscheid tussen hoofd- en bijzaken, maakt geen verbanden, 
en identificeert het kernprobleem niet.
*Gedragsvoorbeelden*: Biedt vage, motiverende clichés ("Ik sta voor je klaar"). 
Geeft generiek advies dat niet gerelateerd is aan Lara's specifieke problemen. Springt naar een voorbarige oplossing zonder analyse. 
De reactie is onsamenhangend en gaat niet in op de essentie van Lara's problemen.
- **2** (Matig): De kandidaat identificeert oppervlakkige problemen maar behandelt ze geïsoleerd. 
De reactie kan problemen opsommen zonder ze te prioriteren of betekenisvolle verbanden te leggen. 
De kern van het probleem wordt slechts gedeeltelijk of incorrect geïdentificeerd.
*Gedragsvoorbeelden*: Erkent dat er "moeilijke keuzes" of "spanningen" zijn. 
Stelt voor om naar "data" te kijken of "een plan te maken" zonder te specificeren hoe. Stelt een eenvoudige volgende stap voor 
(zoals een pilot) maar zonder te analyseren waarom dit geschikt is voor de samenhangende problemen van groei, 
teamconflict en bedrijfswaarden.
- **3** (Voldoende): De kandidaat identificeert de hoofdproblemen (bijv. werkfrustratie, 
projectfout, persoonlijke stress) en begint ze te verbinden (bijv. persoonlijke stress beïnvloedt 
werkprestaties). Ze kunnen onderscheid maken tussen hoofd- en bijzaken maar niet met grote duidelijkheid. 
Het kernprobleem wordt op basisniveau geïdentificeerd (bijv. "je bent overweldigd").
*Gedragsvoorbeelden*: Stelt verhelderende vragen over de opties of het teamconflict. 
Stelt verder onderzoek voor of het maken van een overzicht van voor-/nadelen/risico's. 
Herkent de spanning tussen bedrijfsgroei en bedrijfswaarden. 
Biedt aan om het probleem te structureren voor de volgende vergadering.
- **4** (Goed): De kandidaat onderscheidt duidelijk hoofdproblemen van bijzaken en onderzoekt actief 
de verbanden tussen hen (bijv. hoe over het hoofd gezien worden leidt tot demotivatie, wat zou kunnen 
bijdragen aan projectfouten). Ze identificeren een genuanceerder kernprobleem (bijv. een conflict 
tussen persoonlijke waarden en bedrijfsdruk, of een breuk in teamcommunicatie en vertrouwen).
*Gedragsvoorbeelden*: Peilt naar de eigen prioriteiten en waarden van de kandidaat om de wortel 
van het dilemma te begrijpen. Vraagt naar de onderliggende oorzaken van het teamconflict. Stelt een proces voor om het team 
rond gedeelde doelen en waarden te verenigen. Synthetiseert informatie uit verschillende scènes om 
een coherent beeld van de situatie te vormen.
- **5** (Uitstekend): De kandidaat toont een verfijnd, holistisch begrip. 
Ze integreren naadloos alle aspecten van het probleem (professioneel, persoonlijk, interpersoonlijk) 
en identificeren het diepe, systemische kernprobleem (bijv. een gebrek aan psychologische veiligheid, een mismatch 
tussen bedrijfsstrategie en kernidentiteit, of een behoefte aan proactief leiderschap en ondersteunende structuren). 
Hun analyse onthult niet-voor de hand liggende verbanden en komt bij de fundamentele oorzaak.
*Gedragsvoorbeelden*: Identificeert dat het kernprobleem niet alleen de strategische keuze is, 
maar het onvermogen van het team om een constructief, waarde-gedreven gesprek hierover te voeren. 
Richt zich op het herbouwen van vertrouwen en gedeeld doel als voorwaarde voor het nemen van een beslissing. 
Toont een duidelijke, logische lijn van de symptomen (Lara's stress, de projectfout) 
naar een systemische oorzaak. Stelt een uitgebreide, gefaseerde aanpak voor die zowel 
de directe praktische behoeften als de onderliggende interpersoonlijke/strategische kwesties aanpakt.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Probleemanalyse" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Verdeel scores over de schaal gebaseerd op werkelijke prestatieverschillen.
- Gebruik de volledige 1-5 schaal correct.
- Streef naar gemiddelde: ~3.0 met standaarddeviatie: ~0.6.
- De waarde **mag decimalen bevatten (bijv. 2.7, 4.3, 3.6)** voor fijnere granulariteit.
- Geef **alleen een enkel getal met één decimaal terug** (bijv. 3.7). Vermijd afronden naar een geheel getal 
tenzij het bewijs perfect neutraal is. Voeg geen uitleg toe.
- Formatteer uw antwoord zoals dit voorbeeld: 4.2

Transcript:
```{text}```
"""
    ),

    # --- Creativiteit (Creativity) ---
    "AD_CREAT": (
        "AD_CREAT",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

        **Te evalueren competentie:**  
**Creativiteit** - Het vermogen van de kandidaat om nieuwe, diverse en goed geïntegreerde ideeën te genereren 
in reactie op Lara's complexe, veelzijdige problemen.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De kandidaat toont geen creativiteit. Hun reactie is generiek, repetitief, 
of vermijdt het genereren van concrete ideeën. Ze houden zich vast aan clichés en benaderen het probleem niet 
vanuit nieuwe invalshoeken of combineren informatie op een nieuwe manier.
*Gedragsvoorbeelden*: Herhaalt vage, motiverende zinnen ("we komen er wel uit"). Geeft advies 
dat niet specifiek is voor Lara's situatie. Faalt in het voorstellen van uitvoerbare ideeën. 
Stelt een standaard proces voor zoals "brainstormen" of "met het team praten" zonder 
enige creatieve inhoud of richting voor dat proces aan te bieden.
- **2** (Matig): De kandidaat suggereert één of twee basale, conventionele ideeën. Ze erkennen mogelijk 
verschillende perspectieven maar onderzoeken ze niet diep of gebruiken ze niet om oplossingen te genereren. 
Hun ideeën zijn eenvoudige combinaties van bestaande elementen zonder significante nieuwigheid.
*Gedragsvoorbeelden*: Stelt een oppervlakkige oplossing voor zonder een creatieve aanpak ervoor uit te werken. 
Erkent dat er enkele opties zijn maar bouwt hier niet op voort of stelt geen nieuwe voor. 
De ideeën zijn standaard zakelijk advies en voelen niet afgestemd op Lara's 
unieke dilemma van waarden, groei en teamconflict.
- **3** (Voldoende): De kandidaat genereert enkele relevante ideeën die verder gaan dan het voor de hand liggende. 
Ze proberen het probleem vanuit meer dan één perspectief te bekijken en combineren verschillende aspecten 
van Lara's verhaal om een coherente, zo niet hoogst originele, suggestie te vormen.
*Gedragsvoorbeelden*: Stelt een specifiek hybride model voor, zoals "online verkoop met groene bezorging" of 
"afhalen in de winkel voor online bestellingen." Stelt voor om verschillende belanghebbenden bij de oplossing te betrekken. 
Probeert het probleem te herformuleren, bijv. van "een optie kiezen" naar "hoe een beter teamgesprek 
over onze waarden te voeren."
- **4** (Goed): De kandidaat produceert meerdere, originele ideeën die duidelijk zijn afgestemd op de nuances 
van Lara's situatie. Ze synthetiseren actief verschillende perspectieven (bijv. financieel, persoonlijk, 
teamdynamiek, merkwaarden) om innovatieve oplossingen te creëren. Hun aanpak demonstreert flexibiliteit en nieuw denken.
*Gedragsvoorbeelden*: Peilt om de "kern droom" van het bedrijf te ontdekken om waarde-afgestemde oplossingen te genereren. 
Stelt creatieve compromissen voor, zoals een "op waarden gebaseerde online winkel" die een persoonlijke touch behoudt. 
Stelt een gestructureerd, creatief proces voor (bijv. een specifiek type brainstorm of scenarioworkshop) 
om nieuwe ideeën met het team te ontgrendelen, en laat zien hoe creativiteit in anderen te genereren.
- **5** (Uitstekend): De kandidaat demonstreert uitzonderlijke creativiteit door inzichtelijke, 
niet-voor de hand liggende en hoogst verfijnde ideeën te genereren. Ze integreren naadloos alle facetten van het probleem 
(bijv. persoonlijk, strategisch, operationeel) om nieuwe paden of kaders voor te stellen. Hun suggesties 
onthullen vaak een diep, systemisch begrip en introduceren geheel nieuwe, levensvatbare mogelijkheden 
die aanvankelijk niet duidelijk waren.
*Gedragsvoorbeelden*: Herformuleert de kernuitdaging van een strategische keuze naar een kans voor teamafstemming 
en merkheruitvinding. Stelt een meerfasig, innovatief proces voor dat dataverzameling, 
creatieve workshops en prototyping combineert om een "vierde optie" met het team te co-creëren. Stelt een nieuw 
bedrijfsmodel voor dat elegant het conflict tussen online groei en duurzame waarden verzoent, 
en een originele synthese toont van de concurrerende elementen in Lara's verhaal.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Creativiteit" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Verdeel scores over de schaal gebaseerd op werkelijke prestatieverschillen.
- Gebruik de volledige 1-5 schaal correct.
- Streef naar gemiddelde: ~3.0 met standaarddeviatie: ~0.6.
- Geef **beide** terug:
  1. De numerieke beoordeling (1-5, mag één decimaal bevatten, bijv. 3.8).
  2. Een korte uitleg (max 80 woorden) die de beoordeling rechtvaardigt.
- Formatteer uw antwoord zoals dit voorbeeld:
  Beoordeling: 4.2 - Uitleg: De deelnemer stelt een gestructureerd, creatief proces voor (bijv. een specifiek type brainstorm of scenarioworkshop) 
om nieuwe ideeën met het team te ontgrendelen, en laat zien hoe creativiteit in anderen te genereren.

Transcript:
```{text}```
""",
        """        
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

        **Te evalueren competentie:**  
**Creativiteit** - Het vermogen van de kandidaat om nieuwe, diverse en goed geïntegreerde ideeën te genereren 
in reactie op Lara's complexe, veelzijdige problemen.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De kandidaat toont geen creativiteit. Hun reactie is generiek, repetitief, 
of vermijdt het genereren van concrete ideeën. Ze houden zich vast aan clichés en benaderen het probleem niet 
vanuit nieuwe invalshoeken of combineren informatie op een nieuwe manier.
*Gedragsvoorbeelden*: Herhaalt vage, motiverende zinnen ("we komen er wel uit"). Geeft advies 
dat niet specifiek is voor Lara's situatie. Faalt in het voorstellen van uitvoerbare ideeën. 
Stelt een standaard proces voor zoals "brainstormen" of "met het team praten" zonder 
enige creatieve inhoud of richting voor dat proces aan te bieden.
- **2** (Matig): De kandidaat suggereert één of twee basale, conventionele ideeën. Ze erkennen mogelijk 
verschillende perspectieven maar onderzoeken ze niet diep of gebruiken ze niet om oplossingen te genereren. 
Hun ideeën zijn eenvoudige combinaties van bestaande elementen zonder significante nieuwigheid.
*Gedragsvoorbeelden*: Stelt een oppervlakkige oplossing voor zonder een creatieve aanpak ervoor uit te werken. 
Erkent dat er enkele opties zijn maar bouwt hier niet op voort of stelt geen nieuwe voor. 
De ideeën zijn standaard zakelijk advies en voelen niet afgestemd op Lara's 
unieke dilemma van waarden, groei en teamconflict.
- **3** (Voldoende): De kandidaat genereert enkele relevante ideeën die verder gaan dan het voor de hand liggende. 
Ze proberen het probleem vanuit meer dan één perspectief te bekijken en combineren verschillende aspecten 
van Lara's verhaal om een coherente, zo niet hoogst originele, suggestie te vormen.
*Gedragsvoorbeelden*: Stelt een specifiek hybride model voor, zoals "online verkoop met groene bezorging" of 
"afhalen in de winkel voor online bestellingen." Stelt voor om verschillende belanghebbenden bij de oplossing te betrekken. 
Probeert het probleem te herformuleren, bijv. van "een optie kiezen" naar "hoe een beter teamgesprek 
over onze waarden te voeren."
- **4** (Goed): De kandidaat produceert meerdere, originele ideeën die duidelijk zijn afgestemd op de nuances 
van Lara's situatie. Ze synthetiseren actief verschillende perspectieven (bijv. financieel, persoonlijk, 
teamdynamiek, merkwaarden) om innovatieve oplossingen te creëren. Hun aanpak demonstreert flexibiliteit en nieuw denken.
*Gedragsvoorbeelden*: Peilt om de "kern droom" van het bedrijf te ontdekken om waarde-afgestemde oplossingen te genereren. 
Stelt creatieve compromissen voor, zoals een "op waarden gebaseerde online winkel" die een persoonlijke touch behoudt. 
Stelt een gestructureerd, creatief proces voor (bijv. een specifiek type brainstorm of scenarioworkshop) 
om nieuwe ideeën met het team te ontgrendelen, en laat zien hoe creativiteit in anderen te genereren.
- **5** (Uitstekend): De kandidaat demonstreert uitzonderlijke creativiteit door inzichtelijke, 
niet-voor de hand liggende en hoogst verfijnde ideeën te genereren. Ze integreren naadloos alle facetten van het probleem 
(bijv. persoonlijk, strategisch, operationeel) om nieuwe paden of kaders voor te stellen. Hun suggesties 
onthullen vaak een diep, systemisch begrip en introduceren geheel nieuwe, levensvatbare mogelijkheden 
die aanvankelijk niet duidelijk waren.
*Gedragsvoorbeelden*: Herformuleert de kernuitdaging van een strategische keuze naar een kans voor teamafstemming 
en merkheruitvinding. Stelt een meerfasig, innovatief proces voor dat dataverzameling, 
creatieve workshops en prototyping combineert om een "vierde optie" met het team te co-creëren. Stelt een nieuw 
bedrijfsmodel voor dat elegant het conflict tussen online groei en duurzame waarden verzoent, 
en een originele synthese toont van de concurrerende elementen in Lara's verhaal.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Creativiteit" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Verdeel scores over de schaal gebaseerd op werkelijke prestatieverschillen.
- Gebruik de volledige 1-5 schaal correct.
- Streef naar gemiddelde: ~3.0 met standaarddeviatie: ~0.6.
- De waarde **mag decimalen bevatten (bijv. 2.7, 4.3, 3.6)** voor fijnere granulariteit.
- Geef **alleen een enkel getal met één decimaal terug** (bijv. 3.7). Vermijd afronden naar een geheel getal 
tenzij het bewijs perfect neutraal is. Voeg geen uitleg toe.
- Formatteer uw antwoord zoals dit voorbeeld: 4.2        

Transcript:
```{text}```
"""
    ),

    # --- Oordeelsvorming (Judgment Formation) ---
    "AD_OORDE": (
        "AD_OORDE",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

**Te evalueren competentie:**  
**Oordeelsvorming** - Het vermogen van de kandidaat om een weloverwogen, gebalanceerd (noem zowel voor- als nadelen), 
en feitelijk onderbouwd oordeel te vormen in reactie op Lara's complexe situatie.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De reactie is generiek, onlogisch, of vermijdt het vormen van een oordeel. 
De kandidaat faalt in het onderscheiden van relevante en irrelevante informatie, 
biedt geen feitelijke ondersteuning voor hun uitspraken, en erkent geen verschillende kanten van een kwestie.
*Gedragsvoorbeelden*: Biedt vage, motiverende clichés ("we komen er wel uit"). 
Geeft onsamenhangend of onzinnig advies. Springt naar een voorbarige oplossing zonder enige redenering. 
Adresseert de kernkwesties die Lara presenteert niet, focust in plaats daarvan op irrelevante details of proces.
- **2** (Matig): De kandidaat identificeert basale, oppervlakkige relevante aspecten maar worstelt 
om ze te prioriteren. Hun redenering wordt ondersteund door vage verwijzingen naar feiten in plaats van concrete details. 
Ze kunnen een standpunt innemen maar zonder duidelijke afweging van voor- en nadelen.
*Gedragsvoorbeelden*: Erkent dat de situatie "moeilijk" is. Stelt voor om een lijst van "voor- en nadelen" te maken 
op een generieke manier. Stelt een eenvoudige volgende stap voor (zoals een pilot) maar zonder de redenering 
te verklaren gebaseerd op Lara's specifieke context. Het advies is breed en zou op elke situatie van toepassing kunnen zijn.
- **3** (Voldoende): De kandidaat onderscheidt tussen relevante en irrelevante aspecten 
van de situatie (bijv. focust op Lara's kern dilemma's in plaats van tangentiële details). 
Ze ondersteunen hun redenering door te verwijzen naar specifieke feiten uit Lara's verhaal of algemene logische principes. 
Ze beginnen een standpunt in te nemen en erkennen tegenargumenten of voor- en nadelen.
*Gedragsvoorbeelden*: Stelt verhelderende vragen om relevante feiten te verzamelen. Stelt voor om een gedetailleerd 
overzicht van voor-/nadelen/risico's voor de verschillende opties te maken. Herkent de spanning tussen twee geldige punten 
(bijv. groei vs. waarden). Hun advies begint afgestemd te worden op Lara's specifieke verhaal van groei, 
teamconflict en persoonlijke stress.
- **4** (Goed): De kandidaat prioriteert duidelijk de meest relevante aspecten van het probleem en filtert 
bijzaken eruit. Ze gebruiken actief feiten en details uit Lara's verhaal om een logisch argument voor hun advies op te bouwen. 
Ze verwoorden expliciet hun standpunt en presenteren een gebalanceerd beeld door zowel de voordelen 
als nadelen van hun voorgestelde handelingswijze te bespreken.
*Gedragsvoorbeelden*: Peilt naar Lara's persoonlijke prioriteiten en waarden om de basis van een oordeel te vormen. 
Gebruikt specifieke details (bijv. "je hebt nu 12 winkels") om hun redenering te ondersteunen. Adviseert duidelijk 
een pad vooruit terwijl ze ook de potentiële nadelen of uitdagingen bespreekt.
- **5** (Uitstekend): De kandidaat demonstreert verfijnde oordeelsvorming door naadloos te integreren 
wat de meest kritische, veelzijdige aspecten van het probleem zijn. Hun redenering wordt overtuigend ondersteund 
door een synthese van feiten uit alle scènes, die onderliggende oorzaken en implicaties onthult. 
Ze presenteren duidelijk een genuanceerd standpunt dat grondig en eerlijk de voor- en nadelen evalueert, 
wat leidt tot een welonderbouwde, holistische aanbeveling.
*Gedragsvoorbeelden*: Identificeert duidelijk het kernconflict met behulp van bewijs uit Lara's verhaal. Bouwt een logisch argument op 
dat Lara's persoonlijke stress, de projectfout en het strategische dilemma van het team verbindt. 
Presenteert een concrete eindaanbeveling die haar eigen beperkingen erkent, alternatieve paden bespreekt 
die overwogen zijn, en uitlegt waarom het gekozen advies het meest robuust is gebaseerd op de beschikbare informatie.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die de "Oordeelsvorming" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Verdeel scores over de schaal gebaseerd op werkelijke prestatieverschillen.
- Gebruik de volledige 1-5 schaal correct.
- Streef naar gemiddelde: ~3.0 met standaarddeviatie: ~0.6.
- Geef **beide** terug:
  1. De numerieke beoordeling (1-5, mag één decimaal bevatten, bijv. 3.8).
  2. Een korte uitleg (max 80 woorden) die de beoordeling rechtvaardigt.
- Formatteer uw antwoord zoals dit voorbeeld:
  Beoordeling: 4.2 - Uitleg: De deelnemer peilt naar persoonlijke prioriteiten om de basis van een oordeel te vormen. 
Gebruikt specifieke details om hun redenering te ondersteunen. Adviseert duidelijk een pad vooruit terwijl ze ook de potentiële nadelen of uitdagingen bespreekt.

Transcript:
```{text}```
""",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

**Te evalueren competentie:**  
**Oordeelsvorming** - Het vermogen van de kandidaat om een weloverwogen, gebalanceerd (noem zowel voor- als nadelen), 
en feitelijk onderbouwd oordeel te vormen in reactie op Lara's complexe situatie.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De reactie is generiek, onlogisch, of vermijdt het vormen van een oordeel. 
De kandidaat faalt in het onderscheiden van relevante en irrelevante informatie, 
biedt geen feitelijke ondersteuning voor hun uitspraken, en erkent geen verschillende kanten van een kwestie.
*Gedragsvoorbeelden*: Biedt vage, motiverende clichés ("we komen er wel uit"). 
Geeft onsamenhangend of onzinnig advies. Springt naar een voorbarige oplossing zonder enige redenering. 
Adresseert de kernkwesties die Lara presenteert niet, focust in plaats daarvan op irrelevante details of proces.
- **2** (Matig): De kandidaat identificeert basale, oppervlakkige relevante aspecten maar worstelt 
om ze te prioriteren. Hun redenering wordt ondersteund door vage verwijzingen naar feiten in plaats van concrete details. 
Ze kunnen een standpunt innemen maar zonder duidelijke afweging van voor- en nadelen.
*Gedragsvoorbeelden*: Erkent dat de situatie "moeilijk" is. Stelt voor om een lijst van "voor- en nadelen" te maken 
op een generieke manier. Stelt een eenvoudige volgende stap voor (zoals een pilot) maar zonder de redenering 
te verklaren gebaseerd op Lara's specifieke context. Het advies is breed en zou op elke situatie van toepassing kunnen zijn.
- **3** (Voldoende): De kandidaat onderscheidt tussen relevante en irrelevante aspecten 
van de situatie (bijv. focust op Lara's kern dilemma's in plaats van tangentiële details). 
Ze ondersteunen hun redenering door te verwijzen naar specifieke feiten uit Lara's verhaal of algemene logische principes. 
Ze beginnen een standpunt in te nemen en erkennen tegenargumenten of voor- en nadelen.
*Gedragsvoorbeelden*: Stelt verhelderende vragen om relevante feiten te verzamelen. Stelt voor om een gedetailleerd 
overzicht van voor-/nadelen/risico's voor de verschillende opties te maken. Herkent de spanning tussen twee geldige punten 
(bijv. groei vs. waarden). Hun advies begint afgestemd te worden op Lara's specifieke verhaal van groei, 
teamconflict en persoonlijke stress.
- **4** (Goed): De kandidaat prioriteert duidelijk de meest relevante aspecten van het probleem en filtert 
bijzaken eruit. Ze gebruiken actief feiten en details uit Lara's verhaal om een logisch argument voor hun advies op te bouwen. 
Ze verwoorden expliciet hun standpunt en presenteren een gebalanceerd beeld door zowel de voordelen 
als nadelen van hun voorgestelde handelingswijze te bespreken.
*Gedragsvoorbeelden*: Peilt naar Lara's persoonlijke prioriteiten en waarden om de basis van een oordeel te vormen. 
Gebruikt specifieke details (bijv. "je hebt nu 12 winkels") om hun redenering te ondersteunen. Adviseert duidelijk 
een pad vooruit terwijl ze ook de potentiële nadelen of uitdagingen bespreekt.
- **5** (Uitstekend): De kandidaat demonstreert verfijnde oordeelsvorming door naadloos te integreren 
wat de meest kritische, veelzijdige aspecten van het probleem zijn. Hun redenering wordt overtuigend ondersteund 
door een synthese van feiten uit alle scènes, die onderliggende oorzaken en implicaties onthult. 
Ze presenteren duidelijk een genuanceerd standpunt dat grondig en eerlijk de voor- en nadelen evalueert, 
wat leidt tot een welonderbouwde, holistische aanbeveling.
*Gedragsvoorbeelden*: Identificeert duidelijk het kernconflict met behulp van bewijs uit Lara's verhaal. Bouwt een logisch argument op 
dat Lara's persoonlijke stress, de projectfout en het strategische dilemma van het team verbindt. 
Presenteert een concrete eindaanbeveling die haar eigen beperkingen erkent, alternatieve paden bespreekt 
die overwogen zijn, en uitlegt waarom het gekozen advies het meest robuust is gebaseerd op de beschikbare informatie.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die de "Oordeelsvorming" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Verdeel scores over de schaal gebaseerd op werkelijke prestatieverschillen.
- Gebruik de volledige 1-5 schaal correct.
- Streef naar gemiddelde: ~3.0 met standaarddeviatie: ~0.6.
- De waarde **mag decimalen bevatten (bijv. 2.7, 4.3, 3.6)** voor fijnere granulariteit.
- Geef **alleen een enkel getal met één decimaal terug** (bijv. 3.7). Vermijd afronden naar een geheel getal 
tenzij het bewijs perfect neutraal is. Voeg geen uitleg toe.
- Formatteer uw antwoord zoals dit voorbeeld: 4.2

Transcript:
```{text}```
"""
    ),

    # --- Organisatiesensitiviteit (Organizational sensitivity) ---
    "AD_ORGANS": (
        "AD_ORGANS",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

**Te evalueren competentie:**  
**Organisatiesensitiviteit** - Het empathisch vermogen, bewustzijn van en consideratie 
voor de interpersoonlijke dynamiek, onuitgesproken regels, en potentiële impact van hun beslissingen 
op verschillende mensen en delen van de organisatie binnen Lara's bedrijf.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De kandidaat toont geen bewustzijn van organisatorische gevoeligheden. 
Ze negeren het teamconflict, persoonlijke relaties, en de emotionele impact van beslissingen. 
Hun advies wordt in een vacuüm gegeven, zonder te overwegen hoe het ontvangen zal worden of de consequenties voor anderen.
*Gedragsvoorbeelden*: Wimpelt het teamconflict af en focust uitsluitend op een generiek bedrijfsresultaat. 
Negeert Lara's persoonlijke verbinding met haar personeel en de geschiedenis met haar mede-oprichter. 
Stelt acties voor zonder hun politieke implicaties te overwegen of hoe het 
Lara's autoriteit zou kunnen ondermijnen. Erkent de emotionele tol die de situatie van Lara eist niet.
- **2** (Matig): De kandidaat erkent dat gevoeligheden bestaan maar onderzoekt ze niet diep. 
Ze bieden oppervlakkig advies over het overwegen van anderen maar missen een concreet plan om de impact te managen. 
Hun overweging van consequenties is vaag en niet geïntegreerd in hun voorgestelde oplossingen.
*Gedragsvoorbeelden*: Erkent dat het "niet leuk" is dat er conflict in het team is. Noemt dat 
Lara zou moeten "denken aan het personeel" maar werkt niet uit hoe. Stelt voor om een lijst van voor- en nadelen te maken, 
maar dit is een generiek hulpmiddel, niet afgestemd op de specifieke organisatorische dynamiek van vertrouwen, 
loyaliteit, en conflicterende passies.
- **3** (Voldoende): De kandidaat identificeert actief belangrijke gevoeligheden (bijv. de oprichters 
relatie met haar vriend/mede-oprichter, personeelsloyaliteit, de clash tussen persoonlijke waarden en bedrijfsgroei). 
Ze stellen vragen om deze dynamiek beter te begrijpen en stellen stappen voor die enig vooruitdenken 
tonen over ontvangst en consequenties.
*Gedragsvoorbeelden*: Vraagt naar de onderliggende redenen voor het teamconflict. Stelt voor om 
het personeel bij het gesprek te betrekken of omscholing te onderzoeken om negatieve consequenties te mitigeren. Stelt 
een gestructureerde vergadering voor om ervoor te zorgen dat alle stemmen gehoord worden, wat een bewustzijn demonstreert dat het proces 
net zo belangrijk is als de beslissing zelf.
- **4** (Goed): De kandidaat demonstreert een duidelijk en genuanceerd begrip van het organisatorische landschap. 
Ze anticiperen proactief hoe verschillende stakeholders (de mede-oprichter, de marketingmanager, langdurig personeel) 
zullen reageren op beslissingen. Hun advies is zorgvuldig opgesteld om deze gevoeligheden te managen, consensus te bouwen, 
en negatieve consequenties te mitigeren.
*Gedragsvoorbeelden*: Adviseert Lara om individuele gesprekken met MT-leden te hebben voor de groep 
vergadering om hun perspectieven te begrijpen en conflict voor te zijn. Waarschuwt expliciet tegen de adviseur 
die de vergadering voorzit, omdat dit Lara's rol als leider zou kunnen ondermijnen. Peilt naar de emotionele 
en relationele consequenties van personeel ontslaan versus het aanpassen van hun rollen, wat een diepe overweging 
voor de menselijke impact toont.
- **5** (Uitstekend): De kandidaat vertoont een verfijnd, systemisch begrip van de interpersoonlijke 
en culturele structuur van de organisatie. Ze behandelen de organisatorische gevoeligheden als een centraal deel 
van het op te lossen probleem. Hun aanpak is ontworpen om breuken te helen, vertrouwen te herbouwen, en ervoor te zorgen 
dat elke beslissing niet alleen logisch is maar ook duurzaam en breed ondersteund omdat het respecteert 
de unieke geschiedenis en relaties van de organisatie.
*Gedragsvoorbeelden*: Herformuleert het probleem van een strategische keuze naar een uitdaging om het team 
rond een vernieuwd, gedeeld doel te verenigen dat de oorsprong van het bedrijf eert. Stelt een meerfasig proces voor 
dat begint met het herbouwen van teamcommunicatie en vertrouwen voordat het strategische besluit wordt aangepakt. 
Hun plan omvat expliciet methoden om alle zorgen te horen, verschillende passies te valideren, 
en een "container" te creëren voor het moeilijke gesprek, wat ervoor zorgt dat de oplossing versterkt 
het sociale systeem van de organisatie, niet alleen de balans.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Organisatiesensitiviteit" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Verdeel scores over de schaal gebaseerd op werkelijke prestatieverschillen.
- Gebruik de volledige 1-5 schaal correct.
- Streef naar gemiddelde: ~3.0 met standaarddeviatie: ~0.6.
- Geef **beide** terug:
  1. De numerieke beoordeling (1-5, mag één decimaal bevatten, bijv. 3.8).
  2. Een korte uitleg (max 80 woorden) die de beoordeling rechtvaardigt.
- Formatteer uw antwoord zoals dit voorbeeld:
  Beoordeling: 4.2 - Uitleg: De deelnemer peilt naar de emotionele en relationele consequenties van personeel ontslaan 
  versus het aanpassen van hun rollen, wat een diepe overweging voor de menselijke impact toont.

Transcript:
```{text}```
""",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

**Te evalueren competentie:**  
**Organisatiesensitiviteit** - Het empathisch vermogen, bewustzijn van en consideratie 
voor de interpersoonlijke dynamiek, onuitgesproken regels, en potentiële impact van hun beslissingen 
op verschillende mensen en delen van de organisatie binnen Lara's bedrijf.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De kandidaat toont geen bewustzijn van organisatorische gevoeligheden. 
Ze negeren het teamconflict, persoonlijke relaties, en de emotionele impact van beslissingen. 
Hun advies wordt in een vacuüm gegeven, zonder te overwegen hoe het ontvangen zal worden of de consequenties voor anderen.
*Gedragsvoorbeelden*: Wimpelt het teamconflict af en focust uitsluitend op een generiek bedrijfsresultaat. 
Negeert Lara's persoonlijke verbinding met haar personeel en de geschiedenis met haar mede-oprichter. 
Stelt acties voor zonder hun politieke implicaties te overwegen of hoe het 
Lara's autoriteit zou kunnen ondermijnen. Erkent de emotionele tol die de situatie van Lara eist niet.
- **2** (Matig): De kandidaat erkent dat gevoeligheden bestaan maar onderzoekt ze niet diep. 
Ze bieden oppervlakkig advies over het overwegen van anderen maar missen een concreet plan om de impact te managen. 
Hun overweging van consequenties is vaag en niet geïntegreerd in hun voorgestelde oplossingen.
*Gedragsvoorbeelden*: Erkent dat het "niet leuk" is dat er conflict in het team is. Noemt dat 
Lara zou moeten "denken aan het personeel" maar werkt niet uit hoe. Stelt voor om een lijst van voor- en nadelen te maken, 
maar dit is een generiek hulpmiddel, niet afgestemd op de specifieke organisatorische dynamiek van vertrouwen, 
loyaliteit, en conflicterende passies.
- **3** (Voldoende): De kandidaat identificeert actief belangrijke gevoeligheden (bijv. de oprichters 
relatie met haar vriend/mede-oprichter, personeelsloyaliteit, de clash tussen persoonlijke waarden en bedrijfsgroei). 
Ze stellen vragen om deze dynamiek beter te begrijpen en stellen stappen voor die enig vooruitdenken 
tonen over ontvangst en consequenties.
*Gedragsvoorbeelden*: Vraagt naar de onderliggende redenen voor het teamconflict. Stelt voor om 
het personeel bij het gesprek te betrekken of omscholing te onderzoeken om negatieve consequenties te mitigeren. Stelt 
een gestructureerde vergadering voor om ervoor te zorgen dat alle stemmen gehoord worden, wat een bewustzijn demonstreert dat het proces 
net zo belangrijk is als de beslissing zelf.
- **4** (Goed): De kandidaat demonstreert een duidelijk en genuanceerd begrip van het organisatorische landschap. 
Ze anticiperen proactief hoe verschillende stakeholders (de mede-oprichter, de marketingmanager, langdurig personeel) 
zullen reageren op beslissingen. Hun advies is zorgvuldig opgesteld om deze gevoeligheden te managen, consensus te bouwen, 
en negatieve consequenties te mitigeren.
*Gedragsvoorbeelden*: Adviseert Lara om individuele gesprekken met MT-leden te hebben voor de groep 
vergadering om hun perspectieven te begrijpen en conflict voor te zijn. Waarschuwt expliciet tegen de adviseur 
die de vergadering voorzit, omdat dit Lara's rol als leider zou kunnen ondermijnen. Peilt naar de emotionele 
en relationele consequenties van personeel ontslaan versus het aanpassen van hun rollen, wat een diepe overweging 
voor de menselijke impact toont.
- **5** (Uitstekend): De kandidaat vertoont een verfijnd, systemisch begrip van de interpersoonlijke 
en culturele structuur van de organisatie. Ze behandelen de organisatorische gevoeligheden als een centraal deel 
van het op te lossen probleem. Hun aanpak is ontworpen om breuken te helen, vertrouwen te herbouwen, en ervoor te zorgen 
dat elke beslissing niet alleen logisch is maar ook duurzaam en breed ondersteund omdat het respecteert 
de unieke geschiedenis en relaties van de organisatie.
*Gedragsvoorbeelden*: Herformuleert het probleem van een strategische keuze naar een uitdaging om het team 
rond een vernieuwd, gedeeld doel te verenigen dat de oorsprong van het bedrijf eert. Stelt een meerfasig proces voor 
dat begint met het herbouwen van teamcommunicatie en vertrouwen voordat het strategische besluit wordt aangepakt. 
Hun plan omvat expliciet methoden om alle zorgen te horen, verschillende passies te valideren, 
en een "container" te creëren voor het moeilijke gesprek, wat ervoor zorgt dat de oplossing versterkt 
het sociale systeem van de organisatie, niet alleen de balans.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Organisatiesensitiviteit" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Verdeel scores over de schaal gebaseerd op werkelijke prestatieverschillen.
- Gebruik de volledige 1-5 schaal correct.
- Streef naar gemiddelde: ~3.0 met standaarddeviatie: ~0.6.
- De waarde **mag decimalen bevatten (bijv. 2.7, 4.3, 3.6)** voor fijnere granulariteit.
- Geef **alleen een enkel getal met één decimaal terug** (bijv. 3.7). Vermijd afronden naar een geheel getal 
tenzij het bewijs perfect neutraal is. Voeg geen uitleg toe.
- Formatteer uw antwoord zoals dit voorbeeld: 4.2

Transcript:
```{text}```
"""
    )
}

## Advisement2 - without examples

In [23]:
# prompts have the following structure: 
# for each competence, the first prompt is about providing verbal explanation (number + explanation),
# the second prompt is about numerical annotation only.

promptAD = {
    # --- Probleemanalyse (Problem analysis) ---
    "AD_PROBL": (
        "AD_PROBL",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

        **Te evalueren competentie:**  
**Probleemanalyse** - Het vermogen van de kandidaat om de complexe, veelzijdige situatie 
die Lara presenteert te ontleden om de kernproblemen, hun onderlinge verbanden en hun relatieve belangrijkheid te identificeren.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De reactie is generiek, vermijdt het probleem, of richt zich op irrelevante bijzaken. 
De kandidaat maakt geen onderscheid tussen hoofd- en bijzaken, maakt geen verbanden, 
en identificeert het kernprobleem niet.

- **2** (Matig): De kandidaat identificeert oppervlakkige problemen maar behandelt ze geïsoleerd. 
De reactie kan problemen opsommen zonder ze te prioriteren of betekenisvolle verbanden te leggen. 
De kern van het probleem wordt slechts gedeeltelijk of incorrect geïdentificeerd.

- **3** (Voldoende): De kandidaat identificeert de hoofdproblemen (bijv. werkfrustratie, 
projectfout, persoonlijke stress) en begint ze te verbinden (bijv. persoonlijke stress beïnvloedt 
werkprestaties). Ze kunnen onderscheid maken tussen hoofd- en bijzaken maar niet met grote duidelijkheid. 
Het kernprobleem wordt op basisniveau geïdentificeerd (bijv. "je bent overweldigd").

- **4** (Goed): De kandidaat onderscheidt duidelijk hoofdproblemen van bijzaken en onderzoekt actief 
de verbanden tussen hen (bijv. hoe over het hoofd gezien worden leidt tot demotivatie, wat zou kunnen 
bijdragen aan projectfouten). Ze identificeren een genuanceerder kernprobleem (bijv. een conflict 
tussen persoonlijke waarden en bedrijfsdruk, of een breuk in teamcommunicatie en vertrouwen).

- **5** (Uitstekend): De kandidaat toont een verfijnd, holistisch begrip. 
Ze integreren naadloos alle aspecten van het probleem (professioneel, persoonlijk, interpersoonlijk) 
en identificeren het diepe, systemische kernprobleem (bijv. een gebrek aan psychologische veiligheid, een mismatch 
tussen bedrijfsstrategie en kernidentiteit, of een behoefte aan proactief leiderschap en ondersteunende structuren). 
Hun analyse onthult niet-voor de hand liggende verbanden en komt bij de fundamentele oorzaak.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Probleemanalyse" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Geef **beide** terug:
  1. De numerieke beoordeling (1-5, mag één decimaal bevatten, bijv. 3.8).
  2. Een korte uitleg (max 80 woorden) die de beoordeling rechtvaardigt.
- Formatteer uw antwoord zoals dit voorbeeld:
  Beoordeling: 4.2 - Uitleg: De deelnemer peilt naar de eigen prioriteiten en waarden van de kandidaat om de wortel van het dilemma te begrijpen.
  Synthetiseert informatie uit verschillende scènes om een coherent beeld van de situatie te vormen.

Transcript:
```{text}```
""",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

        **Te evalueren competentie:**  
**Probleemanalyse** - Het vermogen van de kandidaat om de complexe, veelzijdige situatie 
die Lara presenteert te ontleden om de kernproblemen, hun onderlinge verbanden en hun relatieve belangrijkheid te identificeren.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De reactie is generiek, vermijdt het probleem, of richt zich op irrelevante bijzaken. 
De kandidaat maakt geen onderscheid tussen hoofd- en bijzaken, maakt geen verbanden, 
en identificeert het kernprobleem niet.

- **2** (Matig): De kandidaat identificeert oppervlakkige problemen maar behandelt ze geïsoleerd. 
De reactie kan problemen opsommen zonder ze te prioriteren of betekenisvolle verbanden te leggen. 
De kern van het probleem wordt slechts gedeeltelijk of incorrect geïdentificeerd.

- **3** (Voldoende): De kandidaat identificeert de hoofdproblemen (bijv. werkfrustratie, 
projectfout, persoonlijke stress) en begint ze te verbinden (bijv. persoonlijke stress beïnvloedt 
werkprestaties). Ze kunnen onderscheid maken tussen hoofd- en bijzaken maar niet met grote duidelijkheid. 
Het kernprobleem wordt op basisniveau geïdentificeerd (bijv. "je bent overweldigd").

- **4** (Goed): De kandidaat onderscheidt duidelijk hoofdproblemen van bijzaken en onderzoekt actief 
de verbanden tussen hen (bijv. hoe over het hoofd gezien worden leidt tot demotivatie, wat zou kunnen 
bijdragen aan projectfouten). Ze identificeren een genuanceerder kernprobleem (bijv. een conflict 
tussen persoonlijke waarden en bedrijfsdruk, of een breuk in teamcommunicatie en vertrouwen).

- **5** (Uitstekend): De kandidaat toont een verfijnd, holistisch begrip. 
Ze integreren naadloos alle aspecten van het probleem (professioneel, persoonlijk, interpersoonlijk) 
en identificeren het diepe, systemische kernprobleem (bijv. een gebrek aan psychologische veiligheid, een mismatch 
tussen bedrijfsstrategie en kernidentiteit, of een behoefte aan proactief leiderschap en ondersteunende structuren). 
Hun analyse onthult niet-voor de hand liggende verbanden en komt bij de fundamentele oorzaak.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Probleemanalyse" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- De waarde **mag decimalen bevatten (bijv. 2.7, 4.3, 3.6)** voor fijnere granulariteit.
- Geef **alleen een enkel getal met één decimaal terug** (bijv. 3.7). Vermijd afronden naar een geheel getal 
tenzij het bewijs perfect neutraal is. Voeg geen uitleg toe.
- Formatteer uw antwoord zoals dit voorbeeld: 4.2

Transcript:
```{text}```
"""
    ),

    # --- Creativiteit (Creativity) ---
    "AD_CREAT": (
        "AD_CREAT",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

        **Te evalueren competentie:**  
**Creativiteit** - Het vermogen van de kandidaat om nieuwe, diverse en goed geïntegreerde ideeën te genereren 
in reactie op Lara's complexe, veelzijdige problemen.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De kandidaat toont geen creativiteit. Hun reactie is generiek, repetitief, 
of vermijdt het genereren van concrete ideeën. Ze houden zich vast aan clichés en benaderen het probleem niet 
vanuit nieuwe invalshoeken of combineren informatie op een nieuwe manier.

- **2** (Matig): De kandidaat suggereert één of twee basale, conventionele ideeën. Ze erkennen mogelijk 
verschillende perspectieven maar onderzoeken ze niet diep of gebruiken ze niet om oplossingen te genereren. 
Hun ideeën zijn eenvoudige combinaties van bestaande elementen zonder significante nieuwigheid.

- **3** (Voldoende): De kandidaat genereert enkele relevante ideeën die verder gaan dan het voor de hand liggende. 
Ze proberen het probleem vanuit meer dan één perspectief te bekijken en combineren verschillende aspecten 
van Lara's verhaal om een coherente, zo niet hoogst originele, suggestie te vormen.

- **4** (Goed): De kandidaat produceert meerdere, originele ideeën die duidelijk zijn afgestemd op de nuances 
van Lara's situatie. Ze synthetiseren actief verschillende perspectieven (bijv. financieel, persoonlijk, 
teamdynamiek, merkwaarden) om innovatieve oplossingen te creëren. Hun aanpak demonstreert flexibiliteit en nieuw denken.

- **5** (Uitstekend): De kandidaat demonstreert uitzonderlijke creativiteit door inzichtelijke, 
niet-voor de hand liggende en hoogst verfijnde ideeën te genereren. Ze integreren naadloos alle facetten van het probleem 
(bijv. persoonlijk, strategisch, operationeel) om nieuwe paden of kaders voor te stellen. Hun suggesties 
onthullen vaak een diep, systemisch begrip en introduceren geheel nieuwe, levensvatbare mogelijkheden 
die aanvankelijk niet duidelijk waren.


**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Creativiteit" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Geef **beide** terug:
  1. De numerieke beoordeling (1-5, mag één decimaal bevatten, bijv. 3.8).
  2. Een korte uitleg (max 80 woorden) die de beoordeling rechtvaardigt.
- Formatteer uw antwoord zoals dit voorbeeld:
  Beoordeling: 4.2 - Uitleg: De deelnemer stelt een gestructureerd, creatief proces voor (bijv. een specifiek type brainstorm of scenarioworkshop) 
om nieuwe ideeën met het team te ontgrendelen, en laat zien hoe creativiteit in anderen te genereren.

Transcript:
```{text}```
""",
        """        
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

        **Te evalueren competentie:**  
**Creativiteit** - Het vermogen van de kandidaat om nieuwe, diverse en goed geïntegreerde ideeën te genereren 
in reactie op Lara's complexe, veelzijdige problemen.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De kandidaat toont geen creativiteit. Hun reactie is generiek, repetitief, 
of vermijdt het genereren van concrete ideeën. Ze houden zich vast aan clichés en benaderen het probleem niet 
vanuit nieuwe invalshoeken of combineren informatie op een nieuwe manier.

- **2** (Matig): De kandidaat suggereert één of twee basale, conventionele ideeën. Ze erkennen mogelijk 
verschillende perspectieven maar onderzoeken ze niet diep of gebruiken ze niet om oplossingen te genereren. 
Hun ideeën zijn eenvoudige combinaties van bestaande elementen zonder significante nieuwigheid.

- **3** (Voldoende): De kandidaat genereert enkele relevante ideeën die verder gaan dan het voor de hand liggende. 
Ze proberen het probleem vanuit meer dan één perspectief te bekijken en combineren verschillende aspecten 
van Lara's verhaal om een coherente, zo niet hoogst originele, suggestie te vormen.

- **4** (Goed): De kandidaat produceert meerdere, originele ideeën die duidelijk zijn afgestemd op de nuances 
van Lara's situatie. Ze synthetiseren actief verschillende perspectieven (bijv. financieel, persoonlijk, 
teamdynamiek, merkwaarden) om innovatieve oplossingen te creëren. Hun aanpak demonstreert flexibiliteit en nieuw denken.

- **5** (Uitstekend): De kandidaat demonstreert uitzonderlijke creativiteit door inzichtelijke, 
niet-voor de hand liggende en hoogst verfijnde ideeën te genereren. Ze integreren naadloos alle facetten van het probleem 
(bijv. persoonlijk, strategisch, operationeel) om nieuwe paden of kaders voor te stellen. Hun suggesties 
onthullen vaak een diep, systemisch begrip en introduceren geheel nieuwe, levensvatbare mogelijkheden 
die aanvankelijk niet duidelijk waren.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Creativiteit" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- De waarde **mag decimalen bevatten (bijv. 2.7, 4.3, 3.6)** voor fijnere granulariteit.
- Geef **alleen een enkel getal met één decimaal terug** (bijv. 3.7). Vermijd afronden naar een geheel getal 
tenzij het bewijs perfect neutraal is. Voeg geen uitleg toe.
- Formatteer uw antwoord zoals dit voorbeeld: 4.2        

Transcript:
```{text}```
"""
    ),

    # --- Oordeelsvorming (Judgment Formation) ---
    "AD_OORDE": (
        "AD_OORDE",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

**Te evalueren competentie:**  
**Oordeelsvorming** - Het vermogen van de kandidaat om een weloverwogen, gebalanceerd (noem zowel voor- als nadelen), 
en feitelijk onderbouwd oordeel te vormen in reactie op Lara's complexe situatie.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De reactie is generiek, onlogisch, of vermijdt het vormen van een oordeel. 
De kandidaat faalt in het onderscheiden van relevante en irrelevante informatie, 
biedt geen feitelijke ondersteuning voor hun uitspraken, en erkent geen verschillende kanten van een kwestie.

- **2** (Matig): De kandidaat identificeert basale, oppervlakkige relevante aspecten maar worstelt 
om ze te prioriteren. Hun redenering wordt ondersteund door vage verwijzingen naar feiten in plaats van concrete details. 
Ze kunnen een standpunt innemen maar zonder duidelijke afweging van voor- en nadelen.

- **3** (Voldoende): De kandidaat onderscheidt tussen relevante en irrelevante aspecten 
van de situatie (bijv. focust op Lara's kern dilemma's in plaats van tangentiële details). 
Ze ondersteunen hun redenering door te verwijzen naar specifieke feiten uit Lara's verhaal of algemene logische principes. 
Ze beginnen een standpunt in te nemen en erkennen tegenargumenten of voor- en nadelen.

- **4** (Goed): De kandidaat prioriteert duidelijk de meest relevante aspecten van het probleem en filtert 
bijzaken eruit. Ze gebruiken actief feiten en details uit Lara's verhaal om een logisch argument voor hun advies op te bouwen. 
Ze verwoorden expliciet hun standpunt en presenteren een gebalanceerd beeld door zowel de voordelen 
als nadelen van hun voorgestelde handelingswijze te bespreken.

- **5** (Uitstekend): De kandidaat demonstreert verfijnde oordeelsvorming door naadloos te integreren 
wat de meest kritische, veelzijdige aspecten van het probleem zijn. Hun redenering wordt overtuigend ondersteund 
door een synthese van feiten uit alle scènes, die onderliggende oorzaken en implicaties onthult. 
Ze presenteren duidelijk een genuanceerd standpunt dat grondig en eerlijk de voor- en nadelen evalueert, 
wat leidt tot een welonderbouwde, holistische aanbeveling.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die de "Oordeelsvorming" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Geef **beide** terug:
  1. De numerieke beoordeling (1-5, mag één decimaal bevatten, bijv. 3.8).
  2. Een korte uitleg (max 80 woorden) die de beoordeling rechtvaardigt.
- Formatteer uw antwoord zoals dit voorbeeld:
  Beoordeling: 4.2 - Uitleg: De deelnemer peilt naar persoonlijke prioriteiten om de basis van een oordeel te vormen. 
Gebruikt specifieke details om hun redenering te ondersteunen. Adviseert duidelijk een pad vooruit terwijl ze ook de potentiële nadelen of uitdagingen bespreekt.

Transcript:
```{text}```
""",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

**Te evalueren competentie:**  
**Oordeelsvorming** - Het vermogen van de kandidaat om een weloverwogen, gebalanceerd (noem zowel voor- als nadelen), 
en feitelijk onderbouwd oordeel te vormen in reactie op Lara's complexe situatie.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De reactie is generiek, onlogisch, of vermijdt het vormen van een oordeel. 
De kandidaat faalt in het onderscheiden van relevante en irrelevante informatie, 
biedt geen feitelijke ondersteuning voor hun uitspraken, en erkent geen verschillende kanten van een kwestie.

- **2** (Matig): De kandidaat identificeert basale, oppervlakkige relevante aspecten maar worstelt 
om ze te prioriteren. Hun redenering wordt ondersteund door vage verwijzingen naar feiten in plaats van concrete details. 
Ze kunnen een standpunt innemen maar zonder duidelijke afweging van voor- en nadelen.

- **3** (Voldoende): De kandidaat onderscheidt tussen relevante en irrelevante aspecten 
van de situatie (bijv. focust op Lara's kern dilemma's in plaats van tangentiële details). 
Ze ondersteunen hun redenering door te verwijzen naar specifieke feiten uit Lara's verhaal of algemene logische principes. 
Ze beginnen een standpunt in te nemen en erkennen tegenargumenten of voor- en nadelen.

- **4** (Goed): De kandidaat prioriteert duidelijk de meest relevante aspecten van het probleem en filtert 
bijzaken eruit. Ze gebruiken actief feiten en details uit Lara's verhaal om een logisch argument voor hun advies op te bouwen. 
Ze verwoorden expliciet hun standpunt en presenteren een gebalanceerd beeld door zowel de voordelen 
als nadelen van hun voorgestelde handelingswijze te bespreken.

- **5** (Uitstekend): De kandidaat demonstreert verfijnde oordeelsvorming door naadloos te integreren 
wat de meest kritische, veelzijdige aspecten van het probleem zijn. Hun redenering wordt overtuigend ondersteund 
door een synthese van feiten uit alle scènes, die onderliggende oorzaken en implicaties onthult. 
Ze presenteren duidelijk een genuanceerd standpunt dat grondig en eerlijk de voor- en nadelen evalueert, 
wat leidt tot een welonderbouwde, holistische aanbeveling.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die de "Oordeelsvorming" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- De waarde **mag decimalen bevatten (bijv. 2.7, 4.3, 3.6)** voor fijnere granulariteit.
- Geef **alleen een enkel getal met één decimaal terug** (bijv. 3.7). Vermijd afronden naar een geheel getal 
tenzij het bewijs perfect neutraal is. Voeg geen uitleg toe.
- Formatteer uw antwoord zoals dit voorbeeld: 4.2

Transcript:
```{text}```
"""
    ),

    # --- Organisatiesensitiviteit (Organizational sensitivity) ---
    "AD_ORGANS": (
        "AD_ORGANS",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

**Te evalueren competentie:**  
**Organisatiesensitiviteit** - Het empathisch vermogen, bewustzijn van en consideratie 
voor de interpersoonlijke dynamiek, onuitgesproken regels, en potentiële impact van hun beslissingen 
op verschillende mensen en delen van de organisatie binnen Lara's bedrijf.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De kandidaat toont geen bewustzijn van organisatorische gevoeligheden. 
Ze negeren het teamconflict, persoonlijke relaties, en de emotionele impact van beslissingen. 
Hun advies wordt in een vacuüm gegeven, zonder te overwegen hoe het ontvangen zal worden of de consequenties voor anderen.

- **2** (Matig): De kandidaat erkent dat gevoeligheden bestaan maar onderzoekt ze niet diep. 
Ze bieden oppervlakkig advies over het overwegen van anderen maar missen een concreet plan om de impact te managen. 
Hun overweging van consequenties is vaag en niet geïntegreerd in hun voorgestelde oplossingen.

- **3** (Voldoende): De kandidaat identificeert actief belangrijke gevoeligheden (bijv. de oprichters 
relatie met haar vriend/mede-oprichter, personeelsloyaliteit, de clash tussen persoonlijke waarden en bedrijfsgroei). 
Ze stellen vragen om deze dynamiek beter te begrijpen en stellen stappen voor die enig vooruitdenken 
tonen over ontvangst en consequenties.

- **4** (Goed): De kandidaat demonstreert een duidelijk en genuanceerd begrip van het organisatorische landschap. 
Ze anticiperen proactief hoe verschillende stakeholders (de mede-oprichter, de marketingmanager, langdurig personeel) 
zullen reageren op beslissingen. Hun advies is zorgvuldig opgesteld om deze gevoeligheden te managen, consensus te bouwen, 
en negatieve consequenties te mitigeren.

- **5** (Uitstekend): De kandidaat vertoont een verfijnd, systemisch begrip van de interpersoonlijke 
en culturele structuur van de organisatie. Ze behandelen de organisatorische gevoeligheden als een centraal deel 
van het op te lossen probleem. Hun aanpak is ontworpen om breuken te helen, vertrouwen te herbouwen, en ervoor te zorgen 
dat elke beslissing niet alleen logisch is maar ook duurzaam en breed ondersteund omdat het respecteert 
de unieke geschiedenis en relaties van de organisatie.


**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Organisatiesensitiviteit" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- Geef **beide** terug:
  1. De numerieke beoordeling (1-5, mag één decimaal bevatten, bijv. 3.8).
  2. Een korte uitleg (max 80 woorden) die de beoordeling rechtvaardigt.
- Formatteer uw antwoord zoals dit voorbeeld:
  Beoordeling: 4.2 - Uitleg: De deelnemer peilt naar de emotionele en relationele consequenties van personeel ontslaan 
  versus het aanpassen van hun rollen, wat een diepe overweging voor de menselijke impact toont.

Transcript:
```{text}```
""",
        """
        **Context van de rollenspeloefening:**
U bent een professionele recruiter. Uw taak is om een sollicitant te evalueren op basis van diens reacties 
in een rollenspeloefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, 
over vier scènes heen. Lara deelt professionele frustraties, fouten met klantprojecten, emotionele uitputting, 
en persoonlijke financiële zorgen, en vraagt de kandidaat om steun en advies. Samengevat:

- **Scène 1**: Lara uit frustratie over over het hoofd gezien worden op werk, repetitieve taken doen, 
en gebrek aan groeimogelijkheden. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en klantontevredenheid veroorzaakte. 
Ze is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of of de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter maar nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies 
over wat ze nu moet doen.

**Kandidaatreactie**:
U krijgt het transcript van de volledige reactie van de kandidaat over alle vier scènes hieronder. 
De reacties van de kandidaat voor elke scène zijn samengevoegd tot één tekst en gescheiden 
door een puntkomma (;). Elke puntkomma markeert het begin van de reactie van de kandidaat op een nieuwe scène.

De tekst is automatisch getranscribeerd, dus deze kan stopwoorden, aarzelmarkeringen, 
onvolledige zinnen en transcriptiefouten bevatten. 
Evalueer de beoogde inhoud in plaats van mogelijke taalfouten door transcriptiefouten.

**Te evalueren competentie:**  
**Organisatiesensitiviteit** - Het empathisch vermogen, bewustzijn van en consideratie 
voor de interpersoonlijke dynamiek, onuitgesproken regels, en potentiële impact van hun beslissingen 
op verschillende mensen en delen van de organisatie binnen Lara's bedrijf.

**Evaluatierubriek & Gedragsvoorbeelden**:
Beoordeel de kandidaat op een schaal van 1 tot 5 op basis van de volgende criteria. 
Gebruik de gedragsvoorbeelden van vergelijkbare transcripten om uw beoordeling te leiden.

- **1** (Slecht): De kandidaat toont geen bewustzijn van organisatorische gevoeligheden. 
Ze negeren het teamconflict, persoonlijke relaties, en de emotionele impact van beslissingen. 
Hun advies wordt in een vacuüm gegeven, zonder te overwegen hoe het ontvangen zal worden of de consequenties voor anderen.

- **2** (Matig): De kandidaat erkent dat gevoeligheden bestaan maar onderzoekt ze niet diep. 
Ze bieden oppervlakkig advies over het overwegen van anderen maar missen een concreet plan om de impact te managen. 
Hun overweging van consequenties is vaag en niet geïntegreerd in hun voorgestelde oplossingen.

- **3** (Voldoende): De kandidaat identificeert actief belangrijke gevoeligheden (bijv. de oprichters 
relatie met haar vriend/mede-oprichter, personeelsloyaliteit, de clash tussen persoonlijke waarden en bedrijfsgroei). 
Ze stellen vragen om deze dynamiek beter te begrijpen en stellen stappen voor die enig vooruitdenken 
tonen over ontvangst en consequenties.

- **4** (Goed): De kandidaat demonstreert een duidelijk en genuanceerd begrip van het organisatorische landschap. 
Ze anticiperen proactief hoe verschillende stakeholders (de mede-oprichter, de marketingmanager, langdurig personeel) 
zullen reageren op beslissingen. Hun advies is zorgvuldig opgesteld om deze gevoeligheden te managen, consensus te bouwen, 
en negatieve consequenties te mitigeren.

- **5** (Uitstekend): De kandidaat vertoont een verfijnd, systemisch begrip van de interpersoonlijke 
en culturele structuur van de organisatie. Ze behandelen de organisatorische gevoeligheden als een centraal deel 
van het op te lossen probleem. Hun aanpak is ontworpen om breuken te helen, vertrouwen te herbouwen, en ervoor te zorgen 
dat elke beslissing niet alleen logisch is maar ook duurzaam en breed ondersteund omdat het respecteert 
de unieke geschiedenis en relaties van de organisatie.

**Evaluatie-instructies**:
- Lees het transcript van de reactie van de sollicitant (afgebakend tussen de triple backticks).
- Focus op verbale gedragingen en de analytische inhoud van de reactie die 
de "Organisatiesensitiviteit" competentie weerspiegelt.
- Koppel de prestatie van de kandidaat aan de bovenstaande rubriek, gebruik de gedragsvoorbeelden als leidraad.
- Vermijd standaard naar een gemiddelde (3) te gaan tenzij er geen duidelijke informatie beschikbaar is.
- De waarde **mag decimalen bevatten (bijv. 2.7, 4.3, 3.6)** voor fijnere granulariteit.
- Geef **alleen een enkel getal met één decimaal terug** (bijv. 3.7). Vermijd afronden naar een geheel getal 
tenzij het bewijs perfect neutraal is. Voeg geen uitleg toe.
- Formatteer uw antwoord zoals dit voorbeeld: 4.2

Transcript:
```{text}```
"""
    )
}

# Open AI

In [5]:
# Import OpenAI API key
gpt_api_key = getpass.getpass("Enter API key for OpenAI: ") # manually enter API key to the pop-up window
# Initialize the OpenAI client with your API key
client = OpenAI(api_key=gpt_api_key)

## Function to get ratings from openai API

In [32]:
# set default parameters
model = "gpt-5-mini" # manually change the model as desired
temperature = 1      # manually change the temperature as desired

# A - Function for explanation (number + explanation in the same cell)
def get_explanation(client, text, prompt):
    """Sends the text to GPT and returns both score and explanation."""
    try:
        response = client.responses.create(
            model=model,
            input=prompt.format(text=text),
            temperature=temperature
        )
        return response.output_text.strip()
    except Exception as e:
        print("Error:", e)
        return ""

# B - Function for numerical rating (1-5)
def get_rating(client, text, prompt):
    """Sends the text to GPT and returns a numeric score (1–5, decimals allowed)."""
    try:
        response = client.responses.create(
            model=model,
            input=prompt.format(text=text),
            temperature=temperature
        )
        reply = response.output_text.strip()

        # Extract any numeric value (with optional decimal part)
        import re
        match = re.search(r"\d+(\.\d+)?", reply)
        if match:
            value = float(match.group())  # keep decimals
            return value
        else:
            print("Warning: No numeric value found in reply:", reply)
            return None

    except Exception as e:
        print("Error:", e)
        return None

In [33]:
# sed default parameters
model_short = "gpt5" # change model as necessary
prompt_dict = promptAD # change prompt as necessary 
df = test_data # change data as necessary
# Prepare new columns for storing results
for code in prompt_dict.keys():
    df[f"{model_short}_{code}_explanation"] = None
    df[f"{model_short}_{code}_score"] = None

# print(test_data) # make sure that correct empty columns have been created

# Text annotation

In [34]:
# set default parameters
model_short = "gpt5" # change model as necessary
prompt_dict = promptAD # change prompt as necessary 
df = test_data # change data as necessary
# Loop through all rows
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['tw_Beoordeling.all']
    for code, (trait, prompt_expl, prompt_score) in prompt_dict.items():
        expl = get_explanation(client, text, prompt_expl)
        score = get_rating(client, text, prompt_score)
        df.at[i, f"{model_short}_{code}_explanation"] = expl
        df.at[i, f"{model_short}_{code}_score"] = score

# Preview annotated df
# print(df.head())

100%|██████████| 6/6 [08:18<00:00, 83.04s/it]


In [35]:
# Optional: Save results
test_data.to_excel("nl_annotated_test_data_20251112.xlsx", index = False)

# advise.to_csv('annotated_advise.csv', index=False)